# Reconhecimento de texto manuscrito - testando o modelo

<a href="https://colab.research.google.com/github/fabiobento/ocr-sis-emb-2024-2/blob/main/5-OCR_Reconhecimento_de_texto_manuscrito_Testando_o_modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




In [ ]:
try:
    import google.colab
    IN_COLAB = True
    print("Executando no Google Colab")
except ImportError:
    IN_COLAB = False
    print("Não executando no Google Colab")

In [132]:
if IN_COLAB:
    # Baixar imagens para o laboratório.
    !wget https://github.com/fabiobento/ocr-sis-emb-2024-2/raw/refs/heads/main/imagens.zip
    !unzip -n -q imagens.zip

# Importando as bibliotecas

In [ ]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from imutils.contours import sort_contours
import matplotlib.pyplot as plt
if IN_COLAB:
    from google.colab.patches import cv2_imshow
else:
    %pip install imutils==0.5.4
    import imutils

In [134]:
# Definir função para exibição de imagens
def imshow(imagem):
    if IN_COLAB:
        cv2_imshow(imagem)
    else:
        # Exibir a imagem com matplotlib
        if len(imagem.shape) == 2:  # Se a imagem tiver apenas 2 dimensões (níveis de cinza)
            plt.imshow(imagem, cmap='gray')
            plt.axis('off')  # Oculta os eixos
            plt.show()               
        else:
            plt.imshow(imagem)        
            plt.axis('off')  # Oculta os eixos
            plt.show()   

# Carregando a rede neural

In [ ]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    !cp /content/drive/MyDrive/sistemas_embarcados/rede_neural.keras rede_neural.keras

rede_neural = load_model('rede_neural.keras')
  
rede_neural.summary()

# Carregando a imagem de teste

In [ ]:
img = cv2.imread('./Imagens/teste-manuscrito01.jpg')
imshow(img)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
imshow(gray)

# Pré-processamento da imagem

In [ ]:
desfoque = cv2.GaussianBlur(gray, (3,3), 0)
imshow(desfoque)

In [ ]:
adapt_media = cv2.adaptiveThreshold(desfoque, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 9)
imshow(adapt_media)

In [ ]:
inv = 255 - adapt_media
imshow(inv)

In [ ]:
dilatado = cv2.dilate(inv, np.ones((3,3)))
imshow(dilatado)

In [ ]:
bordas = cv2.Canny(desfoque, 40, 150)
imshow(bordas)

In [ ]:
dilatado = cv2.dilate(bordas, np.ones((3,3)))
imshow(dilatado)

# Detecção de contornos

In [143]:
def encontrar_contornos(img):
  conts = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  conts = imutils.grab_contours(conts)
  conts = sort_contours(conts, method='left-to-right')[0]
  return conts

In [144]:
conts = encontrar_contornos(dilatado.copy())

In [ ]:
conts

In [146]:
l_min, l_max = 4, 160
a_min, a_max = 14, 140

In [ ]:
caracteres = []
img_cp = img.copy()
for c in conts:
  #print(c)
  (x, y, w, h) = cv2.boundingRect(c)
  #print(x, y, w, h)
  if (w >= l_min and w <= l_max) and (h >= a_min and h <= a_max):
    roi = gray[y:y+ h, x:x + w]
    #imshow(roi)
    thresh = cv2.threshold(roi, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    imshow(thresh)
    cv2.rectangle(img_cp, (x, y), (x + w, y + h), (255, 100, 0), 2)
imshow(img_cp)

# Processando os caracteres detectados

## Extração ROI


In [148]:
def extra_roi(img):
  roi = img[y:y + h, x:x + w]
  return roi

## Limiarização

In [149]:
def limiarizacao(img):
  thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
  return thresh

## Redimensionamento

In [150]:
def redimensiona_img(img, l, a):
  if l > a:
    redimensionada = imutils.resize(img, width=28)
  else:
    redimensionada = imutils.resize(img, height=28)

  (a, l) = redimensionada.shape
  dX = int(max(0, 28 - l) / 2.0)
  dY = int(max(0, 28 - a) / 2.0)

  preenchida = cv2.copyMakeBorder(redimensionada, top=dY, bottom=dY, right=dX, left=dX, borderType=cv2.BORDER_CONSTANT, value=(0,0,0))
  preenchida = cv2.resize(preenchida, (28, 28))
  return preenchida

In [ ]:
(x, y, w, h) = cv2.boundingRect(conts[6])
print(x, y, w, h)
img_teste = limiarizacao(gray[y:y+h, x:x+w])
imshow(img_teste)
(a, l) = img_teste.shape
print(a, l)
img_teste2 = redimensiona_img(img_teste, l, a)
imshow(img_teste2)
img_teste2.shape

In [ ]:
imshow(cv2.resize(img_teste, (28,28)))

## Normalização

In [153]:
def normalizacao(img):
  img = img.astype('float32') / 255.0
  img = np.expand_dims(img, axis=-1)
  return img

In [ ]:
img_teste2.shape, normalizacao(img_teste2).shape

## Processamento das detecções

In [155]:
def processa_caixa(gray, x, y, w, h):
  roi = extra_roi(gray)
  limiar = limiarizacao(roi)
  (a, l) = limiar.shape
  redimensionada = redimensiona_img(limiar, l, a)
  imshow(redimensionada)
  normalizada = normalizacao(redimensionada)
  caracteres.append((normalizada, (x, y, w, h)))

In [ ]:
for c in conts:
  #print(c)
  (x, y, w, h) = cv2.boundingRect(c)
  if (w >= l_min and w <= l_max) and (h >= a_min and h <= a_max):
    processa_caixa(gray, x, y, w, h)

In [ ]:
caracteres[0]

In [ ]:
caixas = [caixa[1] for caixa in caracteres]
caixas

In [159]:
caracteres = np.array([c[0] for c in caracteres], dtype='float32')

In [ ]:
caracteres

# Reconhecimento dos caracteres

In [161]:
numeros = "0123456789"
letras = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
lista_caracteres = numeros + letras
lista_caracteres = [l for l in lista_caracteres]

In [ ]:
print(lista_caracteres)

In [ ]:
caracteres[0].shape

In [ ]:
caracteres.shape

In [ ]:
previsoes = rede_neural.predict(caracteres)

In [ ]:
previsoes

In [ ]:
previsoes.shape

In [ ]:
caixas

In [ ]:
img_cp = img.copy()
for (previsoes, (x, y, w, h)) in zip(previsoes, caixas):
  i = np.argmax(previsoes)
  #print(i)
  probabilidade = previsoes[i]
  #print(probabilidade)
  caractere = lista_caracteres[i]
  #print(caractere)

  cv2.rectangle(img_cp, (x, y), (x + w, y + h), (255,100,0), 2)
  cv2.putText(img_cp, caractere, (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0,0,255), 2)
  print(caractere, ' -> ', probabilidade * 100)

  imshow(img_cp)

In [170]:
def extrai_roi(img, margem=2):
  roi = img[y - margem:y + h + margem, x - margem:x + w + margem]
  return roi

In [ ]:
conts = encontrar_contornos(dilatado.copy())
caracteres = []
for c in conts:
  (x, y, w, h) = cv2.boundingRect(c)
  if (w >= l_min and w <= l_max) and (h >= a_min and h <= a_max):
    processa_caixa(gray, x, y, w, h)

In [ ]:
caixas = [b[1] for b in caracteres]
caracteres = np.array([c[0] for c in caracteres], dtype="float32")
previsoes = rede_neural.predict(caracteres)

In [ ]:
img_cp = img.copy()
for (previsoes, (x, y, w, h)) in zip(previsoes, caixas):
  i = np.argmax(previsoes)
  probabilidade = previsoes[i]
  caractere = lista_caracteres[i]

  cv2.rectangle(img_cp, (x, y), (x + w, y + h), (255,100,0), 2)
  cv2.putText(img_cp, caractere, (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0,0,255), 2)
  print(caractere, ' -> ', probabilidade * 100)

  imshow(img_cp)

# Outros testes

In [174]:
def preprocessar_img(img):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  desfoque = cv2.GaussianBlur(gray, (3, 3), 7)
  bordas = cv2.Canny(desfoque, 40, 150)
  dilatado = cv2.dilate(bordas, np.ones((3,3)))
  return gray, dilatado

In [175]:
def predicao(previsoes, lista_caracteres):
  i = np.argmax(previsoes)
  probabilidade = previsoes[i]
  caractere = lista_caracteres[i]
  return i, probabilidade, caractere

In [176]:
def desenhar_na_img(img_cp, caractere):
  cv2.rectangle(img_cp, (x, y), (x + w, y + h), (255, 100, 0), 2)
  cv2.putText(img_cp, caractere, (x - 10, y - 10),cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0, 0, 255), 2)

## O problema do 0 e O, 1 e I, 5 e S

In [ ]:
img = cv2.imread('./Imagens/teste_manuscrito02.jpg')
imshow(img)

In [ ]:
gray, img_processada = preprocessar_img(img)
imshow(gray)
imshow(img_processada)

In [ ]:
conts = encontrar_contornos(img_processada.copy())
caracteres = []
for c in conts:
  (x, y, w, h) = cv2.boundingRect(c)
  if (w >= l_min and w <= l_max) and (h >= a_min and h <= a_max):
    processa_caixa(gray, x, y, w, h)

caixas = [b[1] for b in caracteres]
caracteres = np.array([c[0] for c in caracteres], dtype="float32")
previsoes = rede_neural.predict(caracteres)

In [180]:
numeros_2 = "OI234S6789"
letras_2 = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
lista_caracteres_2 = numeros_2 + letras_2
lista_caracteres_2 = [l for l in lista_caracteres_2]

In [ ]:
img_cp = img.copy()
for (previsoes, (x, y, w, h)) in zip(previsoes, caixas):
  i, probabilidade, caractere = predicao(previsoes, lista_caracteres_2)
  desenhar_na_img(img_cp, caractere)
imshow(img_cp)

## O problema do texto não detectado

In [ ]:
img = cv2.imread('./Imagens/teste_manuscrito03.jpg')
imshow(img)

In [ ]:
gray, img_processada = preprocessar_img(img)
imshow(gray)
imshow(img_processada)

In [184]:
# RETR_EXTERNAL

In [ ]:
conts = encontrar_contornos(img_processada.copy())
img_cp = img.copy()
for c in conts:
  (x, y, w, h) = cv2.boundingRect(c)
  #if (w >= l_min and w <= l_max) and (h >= a_min and h <= a_max):
  roi = gray[y:y + h, x:x + w]
  thresh = cv2.threshold(roi, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
  cv2.rectangle(img_cp, (x, y), (x + w, y + h), (255, 100, 0), 2)
imshow(img_cp)

In [ ]:
img.shape

In [ ]:
altura, largura = img.shape[:2]
print(altura, largura)

In [188]:
contornos_tamanho = sorted(conts, key=cv2.contourArea, reverse=True)

In [ ]:
contornos_tamanho

In [ ]:
for c in contornos_tamanho:
  (x, y, w, h) = cv2.boundingRect(c)

  if (w >= (largura / 2)) and (h >= altura / 2):
    corta = 8
    img_cortada = img[y+corta:y + h - corta, x+corta:x + w - corta]
    imshow(img_cortada)

In [ ]:
gray, img_processada = preprocessar_img(img_cortada)
imshow(img_processada)

In [ ]:
conts = encontrar_contornos(img_processada.copy())
caracteres = []
for c in conts:
  (x, y, w, h) = cv2.boundingRect(c)
  if (w >= l_min and w <= l_max) and (h >= a_min and h <= a_max):
    processa_caixa(gray, x, y, w, h)

caixas = [b[1] for b in caracteres]
caracteres = np.array([c[0] for c in caracteres], dtype="float32")

previsoes = rede_neural.predict(caracteres)

img_cp = img_cortada.copy()

# percorre todas as predições e caixas delimitadoras
for (previsoes, (x, y, w, h)) in zip(previsoes, caixas):
  i, probabilidade, caractere = predicao(previsoes, lista_caracteres)
  desenhar_na_img(img_cp, caractere)
imshow(img_cp)